<a href="https://colab.research.google.com/github/jballesterosc/hacking-civico/blob/master/tareas/03_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Para descargar el conjunto de datos
!wget http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

# Para descargar el diccionario de los datos
!wget http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip

# Descomprimimos ambas carpetas y movemos archivos
!unzip datos_abiertos_covid19.zip
!unzip diccionario_datos_covid19.zip
!mv ./diccionario_datos_covid19/*.xlsx .
!mv *COVID19MEXICO.csv COVID19MEXICO.csv

# Eliminamos las carpetas comprimidas
!rm -rf diccionario_datos_covid19
!rm datos_abiertos_covid19.zip
!rm diccionario_datos_covid19.zip

--2020-09-14 04:43:34--  http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 187.210.186.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|187.210.186.146|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32082132 (31M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip’

datos_abiertos_covi 100%[===================>]  30.60M  22.7MB/s    in 1.3s    

2020-09-14 04:43:35 (22.7 MB/s) - ‘datos_abiertos_covid19.zip’ saved [32082132/32082132]

--2020-09-14 04:43:35--  http://epidemiologia.salud.gob.mx/gobmx/salud/datos_abiertos/diccionario_datos_covid19.zip
Resolving epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)... 187.191.75.207
Connecting to epidemiologia.salud.gob.mx (epidemiologia.salud.gob.mx)|187.191.75.207|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ep

In [2]:
import statistics

import pandas as pd
import numpy as np

In [3]:
# Especificamos el nombre del archivo por abrir
nombre_archivo = 'COVID19MEXICO.csv'

# Utilizamos la función de Pandas para cargar un archivo CSV
data = pd.read_csv(nombre_archivo, encoding='latin-1')

In [5]:
nombre_diccionario = 'Descriptores_0419.xlsx'
diccionario = pd.read_excel(nombre_diccionario)

In [6]:
nombre_cat = 'Catalogos_0412.xlsx'
sheets = ['ORIGEN', 'SECTOR', 'SEXO', 'TIPO_PACIENTE', 'SI_NO', 'NACIONALIDAD', 'RESULTADO', 'de ENTIDADES', 'MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(nombre_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

##Perfilamiento

###Descripción General

In [8]:
# dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

Número de filas: 1516588
Número de columnas: 35


In [11]:
# limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [12]:
# calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

Valores nulos en "fecha_actualizacion": 0
Valores nulos en "id_registro": 0
Valores nulos en "origen": 0
Valores nulos en "sector": 0
Valores nulos en "entidad_um": 0
Valores nulos en "sexo": 0
Valores nulos en "entidad_nac": 0
Valores nulos en "entidad_res": 0
Valores nulos en "municipio_res": 0
Valores nulos en "tipo_paciente": 0
Valores nulos en "fecha_ingreso": 0
Valores nulos en "fecha_sintomas": 0
Valores nulos en "fecha_def": 0
Valores nulos en "intubado": 0
Valores nulos en "neumonia": 0
Valores nulos en "edad": 0
Valores nulos en "nacionalidad": 0
Valores nulos en "embarazo": 0
Valores nulos en "habla_lengua_indig": 0
Valores nulos en "diabetes": 0
Valores nulos en "epoc": 0
Valores nulos en "asma": 0
Valores nulos en "inmusupr": 0
Valores nulos en "hipertension": 0
Valores nulos en "otra_com": 0
Valores nulos en "cardiovascular": 0
Valores nulos en "obesidad": 0
Valores nulos en "renal_cronica": 0
Valores nulos en "tabaquismo": 0
Valores nulos en "otro_caso": 0
Valores nulos 

###Limpieza

In [13]:
# convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [16]:
# unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

ValueError: ignored

In [17]:
# unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [18]:
# unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [19]:
catalogues['RESULTADO'].columns

Index(['unnamed:_0', 'unnamed:_1'], dtype='object')

In [21]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

Index(['clave', 'descripción'], dtype='object')

In [23]:
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

KeyError: ignored

In [24]:
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

In [25]:
# unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [28]:
# riesgo de defunción en pacientes con hipertensión
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
NO            False        0.960811
              True         0.039189
SE IGNORA     False        0.872118
              True         0.127882
SI            False        0.834710
              True         0.165290
Name: defuncion, dtype: float64

In [30]:
#casos positivos por entidad
data.entidad_res.value_counts()

CIUDAD DE MÉXICO                   315299
MÉXICO                             185609
GUANAJUATO                          86569
NUEVO LEÓN                          78473
PUEBLA                              63347
TAMAULIPAS                          61704
TABASCO                             60139
COAHUILA DE ZARAGOZA                58845
JALISCO                             58777
VERACRUZ DE IGNACIO DE LA LLAVE     49154
SAN LUIS POTOSÍ                     45815
MICHOACÁN DE OCAMPO                 44079
SONORA                              42876
YUCATÁN                             31743
SINALOA                             31431
BAJA CALIFORNIA                     29958
GUERRERO                            29135
DURANGO                             22561
OAXACA                              22151
CHIHUAHUA                           21544
BAJA CALIFORNIA SUR                 20275
HIDALGO                             20029
AGUASCALIENTES                      19741
TLAXCALA                          

In [31]:
#defunciones por entidad
data[data.defuncion].entidad_res.value_counts()

MÉXICO                             13682
CIUDAD DE MÉXICO                   12038
VERACRUZ DE IGNACIO DE LA LLAVE     4861
PUEBLA                              4636
BAJA CALIFORNIA                     4085
JALISCO                             3918
SINALOA                             3759
NUEVO LEÓN                          3558
SONORA                              3284
GUANAJUATO                          3222
TABASCO                             2936
TAMAULIPAS                          2858
GUERRERO                            2307
COAHUILA DE ZARAGOZA                2215
HIDALGO                             2095
CHIHUAHUA                           2043
MICHOACÁN DE OCAMPO                 1937
QUINTANA ROO                        1927
YUCATÁN                             1822
SAN LUIS POTOSÍ                     1790
OAXACA                              1624
TLAXCALA                            1332
MORELOS                             1256
CHIAPAS                             1221
QUERÉTARO       

In [32]:
# fallecimientos en CDMX (entidad con mayor número de casos confirmados)
data[data.defuncion & (data.entidad_res == "CIUDAD DE MÉXICO")].defuncion.value_counts()

True    12038
Name: defuncion, dtype: int64

In [33]:
# fallecimientos de pacientes ambulatorios
data[data.defuncion & (data.tipo_paciente == "AMBULATORIO")].defuncion.value_counts()

True    9633
Name: defuncion, dtype: int64